### Building an image classification model using very little data  

Based on the tutorial by Francois Chollet @fchollet https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and the workbook by Guillaume Dominici https://github.com/gggdominici/keras-workshop

This tutorial presents several ways to build an image classifier using keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:  

- training a small network from scratch (as a baseline)  
- using the bottleneck features of a pre-trained network  
- fine-tuning the top layers of a pre-trained network  
  
This will lead us to cover the following Keras features:   
  
- fit_generator for training Keras a model using Python data generators  
- ImageDataGenerator for real-time data augmentation  
- layer freezing and model fine-tuning  
- ...and more.  


## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```
Note : for this example we only consider 2x1000 training images and 2x400 testing images among the 2x12500 available.

The github repo includes about 3000 images for this model. The original Kaggle dataset is much larger. The purpose of this demo is to show how you can build models with smaller size datasets. You should be able to improve this model by using more data.

### Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using TensorFlow backend.
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:135] successfully opened CUDA library libcurand.so.8.0 locally


In [2]:
##Updated to Keras 2.0
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using TensorFlow backend.


In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [4]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [5]:
# a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers.
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training

In [7]:
epochs = 30
train_samples = 2048
validation_samples = 832

In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size,)
#About 60 seconds an epoch when using CPU

Epoch 1/30
64/64 [==============================] - 4s - loss: 0.7019 - acc: 0.5312 - val_loss: 0.6602 - val_acc: 0.6430
Epoch 2/30
64/64 [==============================] - 3s - loss: 0.6585 - acc: 0.6201 - val_loss: 0.6155 - val_acc: 0.6599
Epoch 3/30
64/64 [==============================] - 3s - loss: 0.6011 - acc: 0.6802 - val_loss: 0.7000 - val_acc: 0.5829
Epoch 4/30
64/64 [==============================] - 3s - loss: 0.5623 - acc: 0.7124 - val_loss: 0.5634 - val_acc: 0.7007
Epoch 5/30
64/64 [==============================] - 3s - loss: 0.5211 - acc: 0.7598 - val_loss: 0.5417 - val_acc: 0.7103
Epoch 6/30
64/64 [==============================] - 3s - loss: 0.4809 - acc: 0.7632 - val_loss: 0.6005 - val_acc: 0.6863
Epoch 7/30
64/64 [==============================] - 3s - loss: 0.4411 - acc: 0.7954 - val_loss: 0.5165 - val_acc: 0.7440
Epoch 8/30
64/64 [==============================] - 3s - loss: 0.4147 - acc: 0.8145 - val_loss: 0.5743 - val_acc: 0.7224
Epoch 9/30
64/64 [==============

In [9]:
model.save_weights('models/basic_cnn_30_epochs.h5')

In [10]:
#model.save_weights('models_trained/basic_cnn_30_epochs.h5')
#model.load_weights('models_trained/basic_cnn_30_epochs.h5')

If your model successfully runs at one epoch, go back and it for 30 epochs by changing epochs above.  I was able to get to an val_acc of 0.71 at 30 epochs.
A copy of a pretrained network is available in the pretrained folder.

### Evaluating on validation set

Computing loss and accuracy :

In [11]:
model.evaluate_generator(validation_generator, validation_samples)

[1.9269507873731737, 0.72945462740384615]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reach ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs.**

## Data augmentation for improving the model

By applying random transformation to our train set, we artificially enhance our dataset with new unseen images.  
This will hopefully reduce overfitting and allows better generalization capability for our network.

Example of data augmentation applied to a picture:
![Example of data augmentation applied to a picture](pictures/cat_data_augmentation.png)

In [12]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 2048 images belonging to 2 classes.


In [13]:
model.fit_generator(
        train_generator_augmented,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples // batch_size,)

Epoch 1/30
64/64 [==============================] - 8s - loss: 0.6084 - acc: 0.7173 - val_loss: 0.5901 - val_acc: 0.6911
Epoch 2/30
64/64 [==============================] - 7s - loss: 0.5393 - acc: 0.7476 - val_loss: 0.5237 - val_acc: 0.7548
Epoch 3/30
64/64 [==============================] - 7s - loss: 0.5114 - acc: 0.7637 - val_loss: 0.5793 - val_acc: 0.7248
Epoch 4/30
64/64 [==============================] - 7s - loss: 0.4931 - acc: 0.7700 - val_loss: 0.5768 - val_acc: 0.7260
Epoch 5/30
64/64 [==============================] - 7s - loss: 0.4846 - acc: 0.8013 - val_loss: 0.5024 - val_acc: 0.7536
Epoch 6/30
64/64 [==============================] - 7s - loss: 0.4590 - acc: 0.7910 - val_loss: 0.5223 - val_acc: 0.7728
Epoch 7/30
64/64 [==============================] - 7s - loss: 0.4643 - acc: 0.7910 - val_loss: 0.5221 - val_acc: 0.7440
Epoch 8/30
64/64 [==============================] - 7s - loss: 0.4606 - acc: 0.7891 - val_loss: 0.5439 - val_acc: 0.7620
Epoch 9/30
64/64 [==============

In [14]:
model.save_weights('models/augmented_30_epochs.h5')

In [15]:
#model.load_weights('models_trained/augmented_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [16]:
model.evaluate_generator(validation_generator, validation_samples)

[0.54872930947189724, 0.77835787259615385]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 100 epochs :

![Accuracy evolution](pictures/scores_with_dataaugmentation.png)

**Thanks to data-augmentation, the accuracy on the validation set improved to ~80%**

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of data.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](pictures/vgg16_original.png)

#### VGG16 model is available in Keras

In [17]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

### Using the VGG16 model to process samples

In [18]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


This is a long process, so we save the output of the VGG16 once and for all.  

In [19]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [20]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, validation_samples // batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [21]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (train_samples // 2) + [1] * (train_samples // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (validation_samples // 2) + [1] * (validation_samples // 2))

And define and train the custom fully connected neural network :

In [22]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model_top.fit(train_data, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_data, validation_labels))

Train on 2048 samples, validate on 832 samples
Epoch 1/30
2048/2048 [==============================] - 0s - loss: 1.0117 - acc: 0.7373 - val_loss: 0.2824 - val_acc: 0.8846
Epoch 2/30
2048/2048 [==============================] - 0s - loss: 0.3854 - acc: 0.8354 - val_loss: 0.3104 - val_acc: 0.8654
Epoch 3/30
2048/2048 [==============================] - 0s - loss: 0.3131 - acc: 0.8740 - val_loss: 0.3117 - val_acc: 0.8666
Epoch 4/30
2048/2048 [==============================] - 0s - loss: 0.2750 - acc: 0.8916 - val_loss: 0.2966 - val_acc: 0.8750
Epoch 5/30
2048/2048 [==============================] - 0s - loss: 0.2181 - acc: 0.9126 - val_loss: 0.2443 - val_acc: 0.9050
Epoch 6/30
2048/2048 [==============================] - 0s - loss: 0.2031 - acc: 0.9204 - val_loss: 0.2397 - val_acc: 0.9135
Epoch 7/30
2048/2048 [==============================] - 0s - loss: 0.1917 - acc: 0.9209 - val_loss: 0.8980 - val_acc: 0.7212
Epoch 8/30
2048/2048 [==============================] - 0s - loss: 0.1669 - ac

The training process of this small neural network is very fast : ~2s per epoch

In [24]:
model_top.save_weights('models/bottleneck_30_epochs.h5')

### Bottleneck model evaluation

In [25]:
#model_top.load_weights('models/bottleneck_30_epochs.h5)

Loss and accuracy :

In [26]:
model_top.evaluate(validation_data, validation_labels)

 32/832 [>.............................] - ETA: 0s

[0.60475906758354259, 0.90264423076923073]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_with_bottleneck.png)

**We reached a 90% accuracy on the validation after ~1m of training (~20 epochs) and 8% of the samples originally available on the Kaggle competition !**

## Fine-tuning the top layers of a a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [27]:
model_vgg = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [29]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('models/bottleneck_30_epochs.h5')

#model_vgg.add(top_model)
model = Model(inputs = model_vgg.input, outputs = top_model(model_vgg.output))

For fine turning, we only want to train a few layers.  This line will set the first 25 layers (up to the conv block) to non-trainable.

In [30]:
for layer in model.layers[:15]:
    layer.trainable = False

In [31]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [32]:
# prepare data augmentation configuration  . . . do we need this?
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


In [33]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

Epoch 1/30
64/64 [==============================] - 14s - loss: 0.3501 - acc: 0.8999 - val_loss: 0.3482 - val_acc: 0.8990
Epoch 2/30
64/64 [==============================] - 14s - loss: 0.1985 - acc: 0.9268 - val_loss: 0.3713 - val_acc: 0.8954
Epoch 3/30
64/64 [==============================] - 14s - loss: 0.1624 - acc: 0.9336 - val_loss: 0.2705 - val_acc: 0.9111
Epoch 4/30
64/64 [==============================] - 14s - loss: 0.1224 - acc: 0.9521 - val_loss: 0.3526 - val_acc: 0.9050
Epoch 5/30
64/64 [==============================] - 14s - loss: 0.1321 - acc: 0.9502 - val_loss: 0.2383 - val_acc: 0.9231
Epoch 6/30
64/64 [==============================] - 14s - loss: 0.0970 - acc: 0.9624 - val_loss: 0.2955 - val_acc: 0.9135
Epoch 7/30
64/64 [==============================] - 14s - loss: 0.0981 - acc: 0.9644 - val_loss: 0.2939 - val_acc: 0.9243
Epoch 8/30
64/64 [==============================] - 14s - loss: 0.0655 - acc: 0.9761 - val_loss: 0.3388 - val_acc: 0.9219
Epoch 9/30
64/64 [======

In [34]:
model.save_weights('models/finetuning_30epochs_vgg.h5')

In [35]:
model.load_weights('models/finetuning_30epochs_vgg.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [36]:
model.evaluate_generator(validation_generator, validation_samples)

[0.31461367263587958, 0.93137770432692313]

**We reached a 93% accuracy on the validation with 8% of the samples originally available on the Kaggle competition!**